#### Importación de librerías

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.stattools import ccf

#### Definición de funciones

##### Eliminación de la tendencia

In [26]:
def detrend_by_difference(df, columns=None, order=1, apply_log=False):
    """
    Elimina la tendencia de las columnas numericas de una serie temporal calculando la diferencia
    entre cada observacion y la anterior. La diferenciacion puede ser de primer o segundo orden.
    """
    # Hace una copia del df
    df_copy = df.copy()

    # Si no se especifican columnas, se usan todas las columnas numericas
    if columns is None:
        columns = df.select_dtypes(include=np.number).columns.tolist()

    # Itera sobre cada columna en la lista columns
    for col in columns:

        # Si apply_log == True aplica una transformacion logaritmica antes de diferenciar
        if apply_log:
            df_copy[col] = np.log(df_copy[col])
    
        # Calcula la diferencia de primer orden
        elif order == 1:
            new_col_name = f"{col}_1st_order_diff"
            df_copy[new_col_name] = df_copy[col].diff()
        
        # Calcula la diferencia de segundo orden
        elif order == 2:
            new_col_name = f"{col}_2nd_order_diff"
            df_copy[new_col_name] = df_copy[col].diff().diff()
        
        else:
            raise ValueError("El parametro 'order' debe ser 1 o 2")
    
    return df_copy.dropna()

##### Pruebas de estacionariedad

In [38]:
# Codigo adaptado de https://www.statsmodels.org/stable/examples/notebooks/generated/stationarity_detrending_adf_kpss.html#Detrending-by-Differencing

# Test ADF
def adf_test(series, alpha_value=0.05):
    """
    Calcula la prueba de Dickey-Fuller aumentada para evaluar la estacionariedad de la serie.
    H0 = la serie es no estacionaria (tiene una raiz unitaria); H1 = la serie es estacionaria.
    """
    # Calcula el test
    adf_test = adfuller(series)

    # Evalua e imprime los resultados del test de hipotesis
    print("--- ADF Test ---")
    if adf_test[1] <= alpha_value:
        print(f"\nEl p-value ({adf_test[1]}) es menor o igual que {alpha_value}.")
        print('Se rechaza la hipótesis nula (H0). La serie temporal es estacionaria.\n')
    else:
        print(f"\nEl p-value ({adf_test[1]}) es mayor que {alpha_value}.")
        print('No se rechaza la hipótesis nula (H0). La serie temporal es no estacionaria.\n')

# Test KPSS
def kpss_test(series, alpha_value=0.05):
    """
    Calcula la prueba de Kwiatkowski-Phillips-Schmidt-Shin para evaluar la estacionariedad de la serie.
    H0 = la serie es estacionaria; H1 = la serie es no estacionaria (tiene una raiz unitaria)
    """
    # Calcula el test
    kpss_test = kpss(series)

    # Evalua e imprime los resultados del test de hipotesis
    print("--- KPSS Test ---")
    if kpss_test[1] <= alpha_value:
        print(f"\nEl p-value ({kpss_test[1]}) es menor o igual que {alpha_value}.")
        print('Se rechaza la hipótesis nula (H0). La serie temporal es no estacionaria.\n')
    else:
        print(f"\nEl p-value ({kpss_test[1]}) es mayor que {alpha_value}.")
        print('No se rechaza la hipótesis nula (H0). La serie temporal es estacionaria.\n')

##### Cálculo de correlaciones cruzadas

In [27]:
def compute_cross_correlation(independent_series, dependent_series, max_lag, plot=True):
    """
    Calcula la correlación cruzada entre una serie de tiempo independiente y otra
    dependiente para un numero de lags definido.
    """
    # Combina las series usando su indice (fecha) y elimina valores no data
    combined_data = pd.concat([
        independent_series, dependent_series], axis=1
        ).dropna()
    
    # Extrae los nombres de las series para asignarlos a variables
    ind_name = independent_series.name
    dep_name = dependent_series.name
    
    # Define las variables x e y para usar en la funcion de correlacion cruzada (CCF)
    x = combined_data[ind_name]
    y = combined_data[dep_name]

    # Calcula CCF para desfases positivos. Se suma 1 porque el conteo parte del lag 0
    ccf_pos = ccf(x, y, nlags=max_lag+1)
    # Para calcular CCF en desfases negativos se invierte el orden de las variables.
    # Se hace un slice partiendo del lag 1 para no repetir el 0
    ccf_neg = ccf(y, x, nlags=max_lag+1)[1:]

    # Combina los resultados de las CCF, .flip invierte el orden de ccf_neg
    ccf_combined = np.concatenate((np.flip(ccf_neg), ccf_pos))
    # Identifica el indice donde la CCF alcanzo el mayor valor absoluto
    optimal_lag_idx = np.argmax(np.abs(ccf_combined))

    # Crea un array con todos los posibles valores de lag
    lags = np.arange(-max_lag, max_lag + 1)
    # Accede al numero de lag donde se maximiza la CCF
    optimal_lag = lags[optimal_lag_idx]
    # Accede al maximo valor de la CCF
    max_corr = ccf_combined[optimal_lag_idx]

    # Almacena los resultados en un df
    results_df = pd.DataFrame({'lag': lags, 'correlation': ccf_combined})

    # Imprime los resultados
    print("--- Analisis de correlacion cruzada ---")
    print(f'Variable independiente: {ind_name}')
    print(f'Variable dependiente: {dep_name}')
    print(f"Desfases analizados: -{max_lag} a +{max_lag} dias.")
    print(f"La correlacion cruzada maxima es: {max_corr:.4f} en un desfase de {optimal_lag} dias")
    print('\nCorrelaciones maximas:')
    print(results_df.sort_values(by='correlation', key=abs, ascending=False).head(10))
    
    # Imprime un grafico del resultado
    if plot:
        plt.figure(figsize=(12, 6))
        plt.stem(results_df['lag'], results_df['correlation'])
        plt.axhline(0, color='black', linewidth=0.8)
        plt.axvline(optimal_lag, color='gray', linestyle='--', label=f'Desfase optimo ({optimal_lag} dias, corr={max_corr:.2f})')
        
        # Lineas de confianza aproximadas
        n_obs = len(x)
        conf_limit = 1.96 / np.sqrt(n_obs)
        plt.axhline(conf_limit, color='red', linestyle='--', linewidth=0.8, label='Límite de significancia 95%')
        plt.axhline(-conf_limit, color='red', linestyle='--', linewidth=0.8)

        plt.title(f'Correlacion cruzada: {ind_name} vs. {dep_name}')
        plt.xlabel('Desfase (dias)')
        plt.ylabel('Coeficiente de correlacion')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.show()

    return results_df

#### Carga y formateo de datos

In [28]:
# Cargar Datos
piezo_df = pd.read_csv(
    '../data/processed/03_daily/piezo-data_SDH1PS01_daily.csv', 
    parse_dates=['Timestamps'], index_col='Timestamps',
    usecols=['Timestamps', 'depth_m']
)

soil_df = pd.read_csv(
    '../data/processed/03_daily/soil-data_z6-25818_daily.csv', 
    parse_dates=['Timestamps'], index_col='Timestamps',
    usecols=['Timestamps',
             'TEROS12_15cm_water-content_m3/m3', 'TEROS12_30cm_water-content_m3/m3', 'TEROS12_48cm_water-content_m3/m3',
             'TEROS12_15cm_soil-temperature_degree_C', 'TEROS12_30cm_soil-temperature_degree_C', 'TEROS12_48cm_soil-temperature_degree_C']
)

# Diccionarios de mapeo para nombres más limpios y extracción de metadatos
# Formato: Sensor_Profundidad_Variable
soil_rename_map = {
    'TEROS12_15cm_water-content_m3/m3': 'Suelo_15cm_Contenido-agua',
    'TEROS12_30cm_water-content_m3/m3': 'Suelo_30cm_Contenido-agua',
    'TEROS12_48cm_water-content_m3/m3': 'Suelo_48cm_Contenido-agua',
    'TEROS12_15cm_soil-temperature_degree_C': 'Suelo_15cm_Temperatura',
    'TEROS12_30cm_soil-temperature_degree_C': 'Suelo_30cm_Temperatura',
    'TEROS12_48cm_soil-temperature_degree_C': 'Suelo_48cm_Temperatura'
}

# Renombrar columnas
soil_df = soil_df.rename(columns=soil_rename_map)
piezo_df = piezo_df.rename(columns={'depth_m': 'Piezometro_NA_Profundidad-nivel-freatico'})

# Unir datasets
wide_df = pd.merge(piezo_df, soil_df, left_index=True, right_index=True, how='inner')

wide_df

,Piezometro_NA_Profundidad-nivel-freatico,Suelo_48cm_Contenido-agua,Suelo_48cm_Temperatura,Suelo_30cm_Contenido-agua,Suelo_30cm_Temperatura,Suelo_15cm_Contenido-agua,Suelo_15cm_Temperatura
Timestamps,,,,,,,
2024-05-23,0.600837,0.416719,6.629167,0.238385,5.712500,0.212885,3.805208
2024-05-24,0.603679,0.422719,6.571875,0.236448,5.638542,0.211469,3.601042
2024-05-25,0.610170,0.427740,6.476042,0.235844,5.498958,0.210885,3.390625
2024-05-26,0.618615,0.429479,6.363542,0.234406,5.292708,0.210177,3.120833
2024-05-27,0.626767,0.430469,6.213542,0.232833,5.139583,0.209344,3.047917
...,...,...,...,...,...,...,...
2025-07-11,0.519963,0.458000,2.900000,0.353472,1.920833,0.297493,0.300000
2025-07-12,0.526304,0.458000,2.859028,0.352507,1.900000,0.296757,0.300000
2025-07-13,0.530751,0.457910,2.800000,0.351542,1.900000,0.295743,0.300000


In [42]:
# # Transformar a formato largo
# long_df = wide_df.reset_index().melt(id_vars='Timestamps', var_name='metadata', value_name='Valor')

# # Extraer metadatos dividiendo el string por guiones bajos (estructura: Sensor_Profundidad_Variable)
# metadata_split = long_df['metadata'].str.split('_', expand=True)
# long_df['Sensor'] = metadata_split[0]           # Ej: Suelo, Piezometro
# long_df['Profundidad'] = metadata_split[1]  # Ej: 15cm, 30cm, NA
# long_df['Variable'] = metadata_split[2]         # Ej: Contenido-agua, Temperatura

# long_df = long_df.drop('metadata', axis=1)

# # Generacion de un espacio entre el numero y la unidad para etiquetas
# long_df['Profundidad'] = long_df['Profundidad'].str.replace('cm', ' cm').replace('NA', 'NA')

# # Definir etiquetas limpias para los gráficos
# variable_labels = {
#     'Profundidad-nivel-freatico': 'Prof. del nivel freático (m)',
#     'Contenido-agua': 'Contenido de agua (m³/m³)',
#     'Temperatura': 'Temperatura (°C)'
# }
# # Mapeamos para usar en el facet grid
# long_df['Variable'] = long_df['Variable'].map(variable_labels)

# # Orden específico para las profundidades en la leyenda
# soil_depth_order = ['15 cm', '30 cm', '48 cm']
# full_depth_order = ['15 cm', '30 cm', '48 cm', 'NA']

# long_df


#### Eliminación de tendencia

In [41]:
detrended_df = detrend_by_difference(wide_df, order=1)

#### Evaluación de estacionariedad (pre y post detrend)

In [47]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller, kpss
import warnings

# Ignorar advertencias de interpolación de p-value en KPSS
warnings.filterwarnings("ignore")

# 1. Modificamos los tests para que RETORNEN valores en lugar de solo imprimir
def get_adf_results(series, alpha=0.05):
    """Retorna p-value y resultado booleano (True=Estacionaria)"""
    result = adfuller(series.dropna())
    p_value = result[1]
    is_stationary = p_value <= alpha
    return p_value, "Estacionaria" if is_stationary else "No Estacionaria"

def get_kpss_results(series, alpha=0.05):
    """Retorna p-value y resultado booleano (True=Estacionaria)"""
    # nlags='auto' es recomendado para KPSS
    result = kpss(series.dropna(), regression='c', nlags='auto')
    p_value = result[1]
    # En KPSS, H0 es Estacionariedad. Si p < alpha, rechazamos H0 (es NO estacionaria)
    is_stationary = p_value > alpha 
    return p_value, "Estacionaria" if is_stationary else "No Estacionaria"

# 2. Función principal para generar la tabla
def generate_stationarity_table(df, order=1):
    results_data = []
    
    # Obtenemos las versiones diferenciadas usando tu función existente
    # Nota: Tu función devuelve un DF con las originales Y las nuevas columnas
    df_detrended = detrend_by_difference(df, order=order)
    
    # Identificamos las columnas originales (numéricas)
    original_cols = df.select_dtypes(include=np.number).columns.tolist()

    print(f"Procesando {len(original_cols)} variables...")
    
    for col in original_cols:
        # --- A. ANÁLISIS SERIE ORIGINAL ---
        adf_p, adf_res = get_adf_results(df[col])
        kpss_p, kpss_res = get_kpss_results(df[col])
        
        results_data.append({
            'Variable': col,
            'Estado': 'Original',
            #'ADF p-value': round(adf_p, 4),
            'ADF Resultado': adf_res,
            #'KPSS p-value': round(kpss_p, 4),
            'KPSS Resultado': kpss_res
        })
        
        # --- B. ANÁLISIS SERIE DIFERENCIADA ---
        # Construimos el nombre de la columna según tu función detrend_by_difference
        suffix = "_1st_order_diff" if order == 1 else "_2nd_order_diff"
        diff_col_name = f"{col}{suffix}"
        
        if diff_col_name in df_detrended.columns:
            series_diff = df_detrended[diff_col_name]
            
            adf_p_diff, adf_res_diff = get_adf_results(series_diff)
            kpss_p_diff, kpss_res_diff = get_kpss_results(series_diff)
            
            results_data.append({
                'Variable': col,
                'Estado': f'Diferenciada (Orden {order})',
                #'ADF p-value': round(adf_p_diff, 4),
                'ADF Resultado': adf_res_diff,
                #'KPSS p-value': round(kpss_p_diff, 4),
                'KPSS Resultado': kpss_res_diff
            })
            
    # Crear DataFrame final
    summary_df = pd.DataFrame(results_data)
    
    # Opcional: Reordenar columnas para mejor lectura
    summary_df = summary_df[[
        'Variable', 'Estado', 
        #'ADF p-value',
        'ADF Resultado', 
        #'KPSS p-value',
        'KPSS Resultado'
    ]]
    
    return summary_df

# --- EJECUCIÓN ---
# Asumiendo que 'wide_df' y tu función 'detrend_by_difference' ya están cargados en memoria
tabla_resultados = generate_stationarity_table(wide_df, order=1)

# Mostrar la tabla
import IPython.display
print("Resultados de Estacionariedad:")
IPython.display.display(tabla_resultados)

Procesando 7 variables...
Resultados de Estacionariedad:


,Variable,Estado,ADF Resultado,KPSS Resultado
0,Piezometro_NA_Profundidad-nivel-freatico,Original,No Estacionaria,No Estacionaria
1,Piezometro_NA_Profundidad-nivel-freatico,Diferenciada (Orden 1),Estacionaria,Estacionaria
2,Suelo_48cm_Contenido-agua,Original,No Estacionaria,Estacionaria
3,Suelo_48cm_Contenido-agua,Diferenciada (Orden 1),Estacionaria,Estacionaria
4,Suelo_48cm_Temperatura,Original,No Estacionaria,No Estacionaria
5,Suelo_48cm_Temperatura,Diferenciada (Orden 1),Estacionaria,No Estacionaria
6,Suelo_30cm_Contenido-agua,Original,No Estacionaria,No Estacionaria
7,Suelo_30cm_Contenido-agua,Diferenciada (Orden 1),Estacionaria,Estacionaria
8,Suelo_30cm_Temperatura,Original,No Estacionaria,No Estacionaria
9,Suelo_30cm_Temperatura,Diferenciada (Orden 1),Estacionaria,No Estacionaria


#### Tabla comparativa de estacionariedad

#### Cálculo de correlaciones cruzadas